In [ ]:
# update dwave packages
!python -m pip install -U dimod dwave-ocean-sdk dwave-neal

In [ ]:
# imports 

from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler
from dwave.system import LeapHybridSampler
# from dwave.system import FixedEmbeddingComposite

from dwave.system.composites import EmbeddingComposite
from dwave.system import FixedEmbeddingComposite, LazyFixedEmbeddingComposite
from dimod import BQM

from dimod.binary_quadratic_model import BinaryQuadraticModel
import greedy
import dimod
import time
import numpy as np
import re
import matplotlib.pyplot as plt
import pandas as pd
import json


# Run experiment for DW 2000Q QPU

Custom choise of EmbeddingComposite. The choise of embedding does not affect solution greatly for this simple games. Further investigation is needed for larger number of playesr/strategies.

In [ ]:
device  = AwsDevice('.../DW_2000Q_6')
device

In [ ]:
# Constracting BQM for the 2 strategies game

Q = {('p1', 'q1'):-3, ('p1', 'q2'):2, ('p2', 'q2'):-3}
bqm = BinaryQuadraticModel(Q, vartype='BINARY')

# add constraints

bqm.add_linear_equality_constraint([('p1',1), ('p2', 1)], constant=-1, lagrange_multiplier=1)
bqm.add_linear_equality_constraint([('q1',1), ('q2', 1)],  constant=-1, lagrange_multiplier=1)
bqm.add_linear_inequality_constraint([('q1', 2), ('q2', -1)], lb=-1, ub=2, lagrange_multiplier=1, label = 'constr1')
bqm.add_linear_inequality_constraint([('q1', -1), ('q2', 1)], lb=-1, ub=1, lagrange_multiplier=1, label = 'constr2')
bqm.add_linear_inequality_constraint([('p1', 1), ('p2', -1)], lb=-1, ub=1, lagrange_multiplier=1, label = 'constr3')
bqm.add_linear_inequality_constraint([('p1', 1), ('p2', 2)], lb=-1, ub=2, lagrange_multiplier=1, label = 'constr4')

In [ ]:
def plot_sampleset2(sampleset, num_sample):
    '''Plots results of sampling in a bar style.
        Arg: sampleset: a set of samples prodused by DWave's sampler
             num_sample: the number of samples received from DWave's sampler'''
        
    keep_vars =['p1', 'p2', 'q1', 'q2']
    sampleset = dimod.keep_variables(sampleset, keep_vars)
    sampleset = sampleset.aggregate()

    df = sampleset.to_pandas_dataframe()

    strategies = {}
    for i, row in df.iterrows():
        strategy = str(df['p1'][i])+str(df['p2'][i])+str(df['q1'][i])+str(df['q2'][i])
        if len((re.findall('1', strategy))) <=2:
            strategies[strategy]=row['num_occurrences']
    
    myList = strategies.items()
    myList = [('p=(' + item[0][:1]+','+ item[0][1:2]+'), q=('+item[0][2:3]+','+ item[0][3:] +')', item[1]) for item in myList]
    myList = sorted(myList) 
    x, y = zip(*myList) 

    plt.bar(x,y, color='Blue')
    plt.title('Frequency of pure NE points based on ' +str(num_sample) + ' samples')
    plt.xlabel('Pure Strategy')
    plt.ylabel('Frequency')
    plt.savefig('Frequency_2strategiesAdvantage.png', dpi=600)
    plt.show()

  


In [ ]:
#paste the device arn
start = time.time()
emd_comp =  BraketDWaveSampler(device_arn='/...Advantage_system4')
sampleset2 = EmbeddingComposite(emd_comp).sample(bqm, num_reads=5000)
end = time.time()
print('Code execution time [sec]:', end - start)

In [ ]:
sampleset2.first

In [ ]:
plot_sampleset2(sampleset=sampleset2, num_sample=5000)

In [ ]:
def performance(device_arn, num_sample, iter_step):
    '''Samples from DWave's hardware and constracts graph performance.
        Stores the info from sampleset and results of sampling
    Args: num_sample: number of samples from a sampler, [start, end]
          device_arn: AWS devide Arn
          iter_step: step to itterate
    Return: Total time for sampling on QPU.'''
  
    emd_comp =  BraketDWaveSampler(device_arn=device_arn)

    # store in json df the samplesest
    json_storage = {}
    solutions = pd.DataFrame()

    total_runtime = []
    time_inQPU = []
    time_sampling = []
    start = num_sample[0]
    end = num_sample[1]
    
    while start <= end:
        # sample
        sampleset = EmbeddingComposite(emd_comp).sample(bqm, num_reads=start)
        json_storage[start] = sampleset.info
        sampleset.to_pandas_dataframe().to_csv(device_arn[-10:] + 'sampleset.csv', mode='a')
        
        # total time in QPU
        qpu_time = sampleset.info['additionalMetadata']['dwaveMetadata']['timing']['qpuAccessTime']

        qpu_overhead_time = sampleset.info['additionalMetadata']['dwaveMetadata']['timing']['qpuAccessOverheadTime']
        total_postproc = sampleset.info['additionalMetadata']['dwaveMetadata']['timing']['totalPostProcessingTime']
        total_postproc_overhead = sampleset.info['additionalMetadata']['dwaveMetadata']['timing']['postProcessingOverheadTime']

        total_runtime.append(sum([qpu_time, qpu_overhead_time, total_postproc, total_postproc_overhead]))
        time_inQPU.append(qpu_time)
        # time spent to sample n samples.
        time_sampling.append(sampleset.info['additionalMetadata']['dwaveMetadata']['timing']['qpuSamplingTime'])

        start += iter_step
    
    # Writing the info
    file_name = device_arn[-10:] + 'sample_info.json'
    json_file = json.dumps(json_storage)
    with open(file_name, "w") as outfile:
        outfile.write(json_file)
    # Writing the solution sets
    solutions.to_csv(device_arn[-10:]+'sampleset.csv')

    return time_inQPU, time_sampling, total_runtime


In [ ]:
# sampling parameters
start = 500
end= 5001
step = 500

#paste the device arn
time_inQPU2000q, time_sampling2000q, total_runtime2000q = performance(device_arn='.../DW_2000Q_6', num_sample=[start, end], iter_step=step)
time_inQPUadvantage, time_samplingadvantage, total_runtimeadvantage = performance(device_arn='.../Advantage_system4', num_sample=[start, end], iter_step=step)

x = [sample for sample in range(start, end, step) ]



In [ ]:
# plot QPU Time
plt.plot(x, time_inQPU2000q, color='blue', marker='o', label='DWave 2000Q 6')
plt.plot(x, time_inQPUadvantage, color='orange', marker='o', label='DWave Advantage System 4.1')
plt.title('DWave QPU Time')
plt.xlabel('Number of samples')
plt.ylabel('Time, microseconds')
plt.legend(loc="upper left")
plt.savefig('QPU_Time_2strategies.png', dpi=600)
# plt.savefig('QPU_Time_3strategies.png', dpi=600)
plt.show()

In [ ]:
# Plot Total Run time
plt.plot(x, total_runtime2000q, color='blue', marker='o', label='DWave 2000Q 6')
plt.plot(x, total_runtimeadvantage, color='orange', marker='o', label='DWave Advantage System 4.1')
plt.title('DWave Total RunTime')
plt.xlabel('Number of samples')
plt.ylabel('Time, microseconds')
plt.legend(loc="upper left")
plt.savefig('Total_Runtime_2strategies.png', dpi=600)
# plt.savefig('Total_Runtime_3strategies.png', dpi=600)
plt.show()

In [ ]:
# plot total sampling time

plt.plot(x, time_sampling2000q, color='blue', marker='o', label='DWave 2000Q 6')
plt.plot(x, time_samplingadvantage, color='orange', marker='o', label='DWave Advantage System 4.1')
plt.title('DWave Sampling Time')
plt.xlabel('Number of samples')
plt.ylabel('Time, microseconds')
plt.legend(loc="upper left")
plt.savefig('Sampling_Time_2strategies.png', dpi=600)
# plt.savefig('Sampling_Time_3strategies.png', dpi=600)
plt.show()

# Run experiment for DW Advantage QPU

In [ ]:
# Constracting BQM for the 3 strategies game
Q = {('p1', 'q1'):10, ('p1', 'q2'):-10, ('p2', 'q2'):-4, ('p2', 'q3'):-7, ('p3', 'q3'):-10}

bqm = BQM(Q, vartype='BINARY')

# constraints on probability
bqm.add_linear_equality_constraint([('p1',1), ('p2', 1), ('p3', 1)], constant=-1, lagrange_multiplier=10)
bqm.add_linear_equality_constraint([('q1',1), ('q2', 1), ('q3', 1)],  constant=-1, lagrange_multiplier=10)

# inequality constraints
# bqm.add_linear_inequality_constraint([('q1', -2), ('q2', 4), ('q3', 1)], lb=-2, ub=4, lagrange_multiplier=1, label = 'constr1')
# bqm.add_linear_inequality_constraint([('q2', 2), ('q3', 1)], lb=1, ub=2, lagrange_multiplier=1, label = 'constr2')
# bqm.add_linear_inequality_constraint([('q1', -5), ('q2', 12), ('q3', 10)], lb=-5, ub=12, lagrange_multiplier=1, label = 'constr3')
# bqm.add_linear_inequality_constraint([('p1', -2), ('p3', -1)], lb=-2, ub=-1, lagrange_multiplier=1, label = 'constr4')
# bqm.add_linear_inequality_constraint([('p1', -5), ('p2', 1), ('p3', 3)], lb=-5, ub=3, lagrange_multiplier=1, label = 'constr5')
# bqm.add_linear_inequality_constraint([('p1', 5), ('p2', 2), ('p3', 10)], lb=2, ub=10, lagrange_multiplier=1, label = 'constr6')


In [ ]:
# test before run 
#paste the device arn
emd_comp =  BraketDWaveSampler(device_arn='.../DW_2000Q_6')
sampleset3 = EmbeddingComposite(emd_comp).sample(bqm, num_reads=1000)

print("Best solution found: \n",sampleset3)

In [ ]:
def plot_sampleset3(sampleset, num_sample):
    '''Plots results of sampling in a bar style.
        Arg: sampleset: a set of samples prodused by DWave's sampler
             num_sample: the number of samples received from DWave's sampler'''
    
    keep_vars =['p1', 'p2', 'p3', 'q1', 'q2', 'q3']
    sampleset = dimod.keep_variables(sampleset, keep_vars)
    sampleset = sampleset.aggregate()

    df = sampleset.to_pandas_dataframe()

    strategies = {}
    for i, row in df.iterrows():
    #     print(row['num_occurrences'])
    #     break
        strategy = str(df['p1'][i])+str(df['p2'][i])+str(df['p3'][i])+str(df['q1'][i]) +str(df['q2'][i])+str(df['q3'][i])
        if len((re.findall('1', strategy))) <=2:
            strategies[strategy]=row['num_occurrences']

    myList = strategies.items()
    myList = [('p=('+item[0][:1]+','+ item[0][1:2]+','+item[0][2:3]+'), q=('+item[0][3:4]+','+ item[0][4:5]+','+ item[0][5:] +')', item[1]) for item in myList]
    myList = sorted(myList) 
    x, y = zip(*myList) 


#     plt.barh(x,y, color='Blue')
    plt.bar(x,y, color='Blue')
    plt.title('Frequency of pure NE points')
    plt.xlabel('Pure Strategy')
    plt.xticks(rotation = 60, horizontalalignment="right", rotation_mode="anchor") # Rotates X-Axis Ticks by 60-degrees
    plt.ylabel('Frequency')
    plt.savefig('Frequency_3strategiesAdvantage.png', dpi=600, bbox_inches = "tight")
    plt.show()


In [ ]:
#paste the device arn
start = time.time()
emd_comp =  BraketDWaveSampler(device_arn='.../d-wave/Advantage_system4')
sampleset3 = EmbeddingComposite(emd_comp).sample(bqm, num_reads=5000)

end = time.time()
print('Code execution time [sec]:', end - start)


In [ ]:
plot_sampleset3(sampleset=sampleset3, num_sample=5000)

In [ ]:
emd_comp =  BraketDWaveSampler(device_arn='.../d-wave/Advantage_system4')
sampleset3 = EmbeddingComposite(emd_comp).sample(bqm, num_reads=5000)

In [ ]:
plot_sampleset3(sampleset=sampleset3, num_sample=5000)

In [ ]:
# sampling parameters
start = 500
end= 5001
step = 500

#paste the device arn
time_inQPU2000q, time_sampling2000q, total_runtime2000q = performance(device_arn='.../DW_2000Q_6', num_sample=[start, end], iter_step=step)
time_inQPUadvantage, time_samplingadvantage, total_runtimeadvantage = performance(device_arn='.../Advantage_system4', num_sample=[start, end], iter_step=step)

x = [sample for sample in range(start, end, step) ]

# plot QPU Time
plt.plot(x, time_inQPU2000q, color='blue', marker='o', label='DWave 2000Q 6')
plt.plot(x, time_inQPUadvantage, color='orange', marker='o', label='DWave Advantage System 4.1')
plt.title('DWave QPU Time')
plt.xlabel('Number of samples')
plt.ylabel('Time, microseconds')
plt.legend(loc="upper left")
# plt.savefig('QPU_Time_2strategies.png', dpi=600)
plt.savefig('QPU_Time_3strategies.png', dpi=600)
plt.show()

In [ ]:
# Plot Total Run time
plt.plot(x, total_runtime2000q, color='blue', marker='o', label='DWave 2000Q 6')
plt.plot(x, total_runtimeadvantage, color='orange', marker='o', label='DWave Advantage System 4.1')
plt.title('DWave Total RunTime')
plt.xlabel('Number of samples')
plt.ylabel('Time, microseconds')
plt.legend(loc="upper left")
# plt.savefig('Total_Runtime_2strategies.png', dpi=600)
plt.savefig('Total_Runtime_3strategies.png', dpi=600)
plt.show()

In [ ]:
# plot total sampling time

plt.plot(x, time_sampling2000q, color='blue', marker='o', label='DWave 2000Q 6')
plt.plot(x, time_samplingadvantage, color='orange', marker='o', label='DWave Advantage System 4.1')
plt.title('DWave Sampling Time')
plt.xlabel('Number of samples')
plt.ylabel('Time, microseconds')
plt.legend(loc="upper left")
# plt.savefig('Sampling_Time_2strategies.png', dpi=600)
plt.savefig('Sampling_Time_3strategies.png', dpi=600)
plt.show()

# The 8 by 8 Example

In [ ]:
#Constracting QUBO Matrix
Q = {('p1','q1'):-12, ('p1','q2'):-6, ('p1','q3'):-12, ('p1','q4'):-12, ('p1','q5'):-6, ('p1','q6'):-12, ('p1','q7'):-12, ('p1', 'q8'):-6,
     ('p2','q5'):-3, ('p2','q6'):-3, ('p2','q7'):-3, ('p2', 'q8'):-3,
     ('p3','q3'):-12, ('p3','q4'):-12, ('p3','q6'):-12, ('p3','q7'):-12, ('p3', 'q8'):-6,
     ('p4','q4'):-12, ('p4','q5'):-4, ('p4','q6'):-12, ('p4','q7'):-12, ('p4', 'q8'):-12,
     ('p5','q5'):-12, ('p5','q6'):-12, ('p5','q7'):-12, ('p5', 'q8'):-12,
     ('p6','q6'):-12, ('p6','q7'):-12, ('p6', 'q8'):-12,
     ('p7','q7'):-12, ('p7', 'q8'):-12,
     ('p8', 'q8'):-12}

bqm = BQM(Q, vartype='BINARY')

# constraints on probability
bqm.add_linear_equality_constraint([('p1',1), ('p2', 1), ('p3', 1), ('p4',1), ('p5', 1), ('p6', 1), ('p7',1), ('p8', 1)], constant=-1, lagrange_multiplier=10)
bqm.add_linear_equality_constraint([('q1',1), ('q2', 1), ('q3', 1), ('q4',1), ('q5', 1), ('q6', 1), ('q7',1), ('q8', 1)],  constant=-1, lagrange_multiplier=10)

# # inequality constraints
bqm.add_linear_inequality_constraint([('q1',2), ('q2',3), ('q3',2),('q4',2), ('q5', 3), ('q6', 2), ('q7', 2), ('q8', 3)], lb=2, ub=3, lagrange_multiplier=1, label = 'constr1')
bqm.add_linear_inequality_constraint([('q1',-2), ('q5', -1), ('q6', -1), ('q7', -1), ('q8', -2)], lb=-2, ub=0, lagrange_multiplier=1, label = 'constr2')
bqm.add_linear_inequality_constraint([('q1',2), ('q3',2),('q4',2), ('q6', 2), ('q7', 2), ('q8', -1)], lb=-1, ub=2, lagrange_multiplier=1, label = 'constr3')
bqm.add_linear_inequality_constraint([('q1',3), ('q3',3),('q4',3), ('q5', 1), ('q6', 3), ('q7', 3), ('q8', 3)], lb=0, ub=3, lagrange_multiplier=1, label = 'constr4')
bqm.add_linear_inequality_constraint([('q1',-6), ('q2',9),('q4',4), ('q5', 12), ('q6', 12), ('q7', 12), ('q8', 12)], lb=-6, ub=12, lagrange_multiplier=1, label = 'constr5')
bqm.add_linear_inequality_constraint([('q1',4), ('q2',3), ('q3',4),('q4',4), ('q5', 4), ('q6', 4), ('q7', 4), ('q8', 4)], lb=3, ub=4, lagrange_multiplier=1, label = 'constr6')
bqm.add_linear_inequality_constraint([('q1',4), ('q2',3), ('q3',4),('q4',4), ('q5', 4), ('q6', 4), ('q7', 4), ('q8', 4)], lb=3, ub=4, lagrange_multiplier=1, label = 'constr7')
bqm.add_linear_inequality_constraint([('q1',-1), ('q2',3), ('q3',3),('q4',2), ('q5', 2), ('q6', 2), ('q7', 2), ('q8', 2)], lb=-1, ub=4, lagrange_multiplier=1, label = 'constr8')

bqm.add_linear_inequality_constraint([('p1',2), ('p2',-1), ('p3',2), ('p4',2), ('p5',-1), ('p6',2), ('p7',2), ('p8',-1)], lb=-1, ub=2, lagrange_multiplier=1, label = 'constr9')
bqm.add_linear_inequality_constraint([('p1',2), ('p5',1), ('p6',1), ('p7',1), ('p8',2)], lb=0, ub=2, lagrange_multiplier=1, label = 'constr10')
bqm.add_linear_inequality_constraint([('p1',2), ('p3',2), ('p4',2), ('p6',2), ('p7',2), ('p8',3)], lb=0, ub=3, lagrange_multiplier=1, label = 'constr11')
bqm.add_linear_inequality_constraint([('p1',2), ('p3',2), ('p4',2), ('p5',1), ('p6',2), ('p7',2), ('p8',2)], lb=0, ub=2, lagrange_multiplier=1, label = 'constr12')
bqm.add_linear_inequality_constraint([('p1',18), ('p2',-3), ('p4',2), ('p5',12), ('p6',12), ('p7',12), ('p8',12)], lb=-3, ub=18, lagrange_multiplier=1, label = 'constr13')
bqm.add_linear_inequality_constraint([('p1',4), ('p2',-1), ('p3',4), ('p4',4), ('p5',4), ('p6',4), ('p7',4), ('p8',4)], lb=-1, ub=4, lagrange_multiplier=1, label = 'constr14')
bqm.add_linear_inequality_constraint([('p1',4), ('p2',-1), ('p3',4), ('p4',4), ('p5',4), ('p6',4), ('p7',4), ('p8',4)], lb=-1, ub=4, lagrange_multiplier=1, label = 'constr15')
bqm.add_linear_inequality_constraint([('p1',3), ('p2',-1), ('p3',-1), ('p4',2), ('p5',2), ('p6',2), ('p7',2), ('p8',2)], lb=-1, ub=3, lagrange_multiplier=1, label = 'constr16')



In [ ]:
def plot_sampleset_8by8(sampleset, num_sample, num_try):
    '''Plots results of sampling in a bar style.
        Arg: sampleset: a set of samples prodused by DWave's sampler
             num_sample: the number of samples received from DWave's sampler'''
    
    keep_vars =['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'q1', 'q2', 'q3',
                'q4', 'q5', 'q6', 'q7', 'q8']
    sampleset = dimod.keep_variables(sampleset, keep_vars)
    sampleset = sampleset.aggregate()

    df = sampleset.to_pandas_dataframe()

    strategies = {}
    for i, row in df.iterrows():
        strategy = str(df['p1'][i])+str(df['p2'][i])+str(df['p3'][i]) \
                  +str(df['p4'][i])+str(df['p5'][i])+str(df['p6'][i]) \
                  +str(df['p7'][i])+str(df['p8'][i])+str(df['q1'][i]) \
                  +str(df['q2'][i])+str(df['q3'][i])+ str(df['q4'][i])+str(df['q5'][i]) \
                  +str(df['q6'][i])+str(df['q7'][i])+str(df['q8'][i])
        if len((re.findall('1', strategy))) <=2:
            strategies[strategy]=row['num_occurrences']
    
    # drop lower that 50
    strategies = {key:value for key, value in strategies.items() if value >= 50 }
    
    myList = strategies.items()
    myList = [('p=('+item[0][:8]+'), q=('+item[0][8:] +')', item[1]) for item in myList]
    # print('myList =', myList)
    myList = sorted(myList) 
    x, y = zip(*myList) 

    plt.figure(figsize=(30,10))
    plt.margins(x=0.01)
    plt.bar(x,y, color='Blue')
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.title('Frequency of pure NE points based on ' +str(num_sample) + ' samples', fontsize=20)
    plt.xlabel('Pure Strategy', fontsize=20)
    plt.xticks(rotation = 90, horizontalalignment="right", rotation_mode="anchor")
    plt.ylabel('Frequency', fontsize=20)
    plt.tight_layout()
    plt.savefig('Frequency_8strategiesAdvantage_'+str(num_try)+'.png', dpi=600, bbox_inches = "tight")
    plt.savefig('Frequency_8strategiesAdvantage_'+str(num_try)+'.pdf', dpi=600, bbox_inches = "tight")
    plt.show()



In [ ]:
# !!!!!!!!! test before run 
#paste the device arn

# emd_comp =  BraketDWaveSampler(device_arn='.../DW_2000Q_6')
emd_comp =  BraketDWaveSampler(device_arn='.../Advantage_system4')
# timing
t_initial = time.time()
sampleset8 = EmbeddingComposite(emd_comp).sample(bqm, num_reads=5000, annealing_time=100)
t_final  = time.time()

print("Best solution found: \n", sampleset8)

In [ ]:
#pass parameters before running
num_try = 22 # store the outputs based on tries to select the best possible later
num_sample=5000


plot_sampleset_8by8(sampleset8, num_sample=num_sample, num_try=num_try)

In [ ]:
print(sampleset8.info['additionalMetadata']['dwaveMetadata']['timing']) 

In [ ]:


# postprocessed = greedy.SteepestDescentSampler().sample(bqm, initial_states=sampleset8)
postprocessed = greedy.SteepestDescentSolver().sample(bqm, initial_states=sampleset8)


In [ ]:
print(postprocessed)

In [ ]:
num_try=23
plot_sampleset_8by8(postprocessed, num_sample=5000, num_try=num_try)

In [ ]:
#paste the device arn
start = time.time()
emd_comp =  BraketDWaveSampler(device_arn='.../DW_2000Q_6')
sampleset8 = EmbeddingComposite(emd_comp).sample(bqm, num_reads=5000)

end = time.time()
print('Code execution time [sec]:', end - start)


In [ ]:
# plotting
plot_sampleset_8by8(sampleset=sampleset8, num_sample=5000)

In [ ]:
#paste the device arn
start = time.time()
emd_comp =  BraketDWaveSampler(device_arn='.../Advantage_system4')
sampleset8 = EmbeddingComposite(emd_comp).sample(bqm, num_reads=5000)

end = time.time()
print('Code execution time [sec]:', end - start)


In [ ]:
# plotting
plot_sampleset_8by8(sampleset=sampleset8, num_sample=5000)

In [ ]:
# sampling parameters
start = 500
end= 5001
step = 500

#paste the device arn
time_inQPU2000q, time_sampling2000q, total_runtime2000q = performance(device_arn='.../DW_2000Q_6', num_sample=[start, end], iter_step=step)
time_inQPUadvantage, time_samplingadvantage, total_runtimeadvantage = performance(device_arn='.../Advantage_system4', num_sample=[start, end], iter_step=step)

x = [sample for sample in range(start, end, step) ]

In [ ]:
# plot QPU Time
plt.plot(x, time_inQPU2000q, color='blue', marker='o', label='DWave 2000Q 6')
plt.plot(x, time_inQPUadvantage, color='orange', marker='o', label='DWave Advantage System 4.1')
plt.title('DWave QPU Time')
plt.xlabel('Number of samples')
plt.ylabel('Time, microseconds')
plt.legend(loc="upper left")
plt.savefig('QPU_Time_8strategies.png', dpi=600)
plt.savefig('QPU_Time_8strategies.pdf', dpi=600)
plt.show()

In [ ]:
# Plot Total Run time
plt.plot(x, total_runtime2000q, color='blue', marker='o', label='DWave 2000Q 6')
plt.plot(x, total_runtimeadvantage, color='orange', marker='o', label='DWave Advantage System 4.1')
plt.title('DWave Total RunTime')
plt.xlabel('Number of samples')
plt.ylabel('Time, microseconds')
plt.legend(loc="upper left")
plt.savefig('Total_Runtime_8strategies.png', dpi=600)
plt.savefig('Total_Runtime_8strategies.pdf', dpi=600)
plt.show()

In [ ]:
# plot total sampling time

plt.plot(x, time_sampling2000q, color='blue', marker='o', label='DWave 2000Q 6')
plt.plot(x, time_samplingadvantage, color='orange', marker='o', label='DWave Advantage System 4.1')
plt.title('DWave Sampling Time')
plt.xlabel('Number of samples')
plt.ylabel('Time, microseconds')
plt.legend(loc="upper left")
plt.savefig('Sampling_Time_8strategies.png', dpi=600)
plt.savefig('Sampling_Tim_8strategies.pdf', dpi=600)
plt.show()